## Non Maximum Supression for object detection


In [7]:
# import necessary packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
def non_max_suppression_slow(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]


    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list, add the index
        # value to the list of picked indexes, then initialize
        # the suppression list (i.e. indexes that will be deleted)
        # using the last index
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        suppress = [last]

        # loop over all indexes in the indexes list
        for pos in range(0, last):
            # grab the current index
            j = idxs[pos]

            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])

            # compute the width and height of the bounding box
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)

            # compute the ratio of overlap between the computed
            # bounding box and the bounding box in the area list
            overlap = float(w * h) / area[j]

            # if there is sufficient overlap, suppress the
            # current bounding box
            if overlap > overlapThresh:
                suppress.append(pos)

        # delete all indexes from the index list that are in the
        # suppression list
        idxs = np.delete(idxs, suppress)

    # return only the bounding boxes that were picked
    return boxes[pick]

In [11]:
# construct a list containing the images that will be examined
# along with their respective bounding boxes
# This is a sample list of images. Own images can be annotated using LabelImg tool (freely available for linux)


images = [
    ("images/lion.jpg", np.array([
    (489, 76, 902, 444),
    (513, 93, 954, 438),
    (447, 75, 973, 456)])),
    ("images/dog.jpg", np.array([
    (24, 31, 178, 179),
    (29, 38, 171, 164),
    (17, 24, 175, 169),
    (5, 35, 189, 164)]))
]


# loop over the images
for (imagePath, boundingBoxes) in images:
    # load the image and clone it
    print("Initial bounding boxes: {}".format(len(boundingBoxes)))
    image = cv2.imread(imagePath)
    orig = image.copy()

    # loop over the bounding boxes for each image and draw them
    for (startX, startY, endX, endY) in boundingBoxes:
        cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 0, 255), 2)

    # perform non-maximum suppression on the bounding boxes
    pick = non_max_suppression_slow(boundingBoxes, 0.3)
    print("After applying non-maximum, bounding boxes: {}".format(len(pick)))

    # loop over the picked bounding boxes and draw them
    for (startX, startY, endX, endY) in pick:
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
        cv2.imwrite(imagePath.split('.')[0]+'_orig.jpg', orig)
        cv2.imwrite(imagePath.split('.')[0]+'_nms.jpg', image)
        print('Images saved in :{}\n'.format(imagePath.split('/')[0]))

Initial bounding boxes: 3
After applying non-maximum, bounding boxes: 1
Images saved in :images

Initial bounding boxes: 4
After applying non-maximum, bounding boxes: 1
Images saved in :images

